<a href="https://colab.research.google.com/github/KellyDRasmussen/KellyDRasmussen/blob/main/dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initialising

In [4]:
#!pip install dash
#!pip install jupyter-dash
#!pip install pyngrok

In [33]:
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
from pyngrok import ngrok

In [7]:
from google.colab import drive
drive.mount('/content/drive')

df=pd.read_csv('/content/drive/MyDrive/Teamcheese/sales_time_series.csv',encoding='utf-8')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data clean up

In [14]:
# List of ShopTypes to drop
brands_to_drop = ['No National Head Customer P/f Poul', 'No National Head Customer Poul Mich', 'Greenland others', 'Island Others', 'Dagrofa Logistik  Grønland', '#', 'Reitan Greenland', 'No National Head Customer', 'Coop Greenland']

# Drop rows with unwanted ShopTypes
df_filtered = df[~df['ShopType'].isin(brands_to_drop)]
df_filtered['ShopType'].unique()

array(['Grossister.', 'Foodservice Danmark', 'Mælkegrossister',
       'Dansk Cater', 'Foodservice øvrige', 'Hørkram Schulz Food Service',
       'Dagrofa', 'Reitan Distribution A/S', 'Salling Group',
       'Aldi Holding ApS (DK)', 'Coop Norden DK', ' Arla Foods',
       'Pizzagrossisterne', 'Lidl Denmark', 'DBE others', 'Premium',
       'MBB'], dtype=object)

In [15]:
#I don't need SoldTo
df = df_filtered.drop(columns=['SoldTo'])
df.columns

Index(['Brand', 'Product', 'Description', 'ShopType', 'City', 'Latitude',
       'Longitude', 'Month', 'Sales_Kg'],
      dtype='object')

In [21]:
#defining segment categories
consumer = ['Dagrofa', 'Reitan Distribution A/S', 'Salling Group', 'Aldi Holding ApS (DK)', 'Coop Norden DK', ' Arla Foods', 'Lidl Denmark']
catering = ['Grossister.', 'Foodservice Danmark', 'Mælkegrossister', 'Dansk Cater', 'Foodservice øvrige', 'Hørkram Schulz Food Service', 'Pizzagrossisterne', 'Premium']
unknown = ['DBE others', 'MBB']

In [22]:
# Create a dictionary mapping old values to new values
value_map = {**dict.fromkeys(consumer, 'Consumer'),
             **dict.fromkeys(catering, 'Catering'),
             **dict.fromkeys(unknown, 'Unknown')}

# Replace values in 'ShopType' column
df['ShopType'] = df['ShopType'].replace(value_map)

In [29]:
# Rename 'Month' column to 'Date'
df = df.rename(columns={'Month': 'Date'})

print(df)


          Brand               Product                 Description  ShopType  \
0      Castello  Crm Cheese Decorated  Castello pineapple 1kg INT  Catering   
1      Castello  Crm Cheese Decorated  Castello pineapple 1kg INT  Catering   
2      Castello  Crm Cheese Decorated  Castello pineapple 1kg INT  Catering   
3      Castello  Crm Cheese Decorated  Castello pineapple 1kg INT  Catering   
4      Castello  Crm Cheese Decorated  Castello pineapple 1kg INT  Catering   
...         ...                   ...                         ...       ...   
88399     Unika      YC Whole Hushåll  Unika Koryfé 1x1,1 kg (DK)  Catering   
88400     Unika      YC Whole Hushåll  Unika Koryfé 1x1,1 kg (DK)  Catering   
88401     Unika      YC Whole Hushåll  Unika Koryfé 1x1,1 kg (DK)  Catering   
88402     Unika      YC Whole Hushåll  Unika Koryfé 1x1,1 kg (DK)  Catering   
88403     Unika      YC Whole Hushåll  Unika Koryfé 1x1,1 kg (DK)  Catering   

             City  Latitude  Longitude        Date 

# Dashboard

In [38]:
selected_brand = 'Unika'
df = df[df['Brand'] == selected_brand]

In [40]:
# Convert Date to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Initialize the Dash app
app = Dash(__name__)

# Layout of the app
app.layout = html.Div([
    dcc.Dropdown(
        id='data-slicer',
        options=[
            {'label': 'Consumer', 'value': 'Consumer'},
            {'label': 'Catering', 'value': 'Catering'}
        ],
        value='Consumer'
    ),
    dcc.Graph(id='time-series'),
    dcc.Graph(id='product-sales'),
    dcc.Graph(id='sales-locations')
])

# Callback to update graphs
@app.callback(
    [Output('time-series', 'figure'),
     Output('product-sales', 'figure'),
     Output('sales-locations', 'figure')],
    [Input('data-slicer', 'value')]
)
def update_graphs(selected_segment):
    filtered_df = df[df['ShopType'] == selected_segment]

    # Debug: Print the filtered DataFrame
    print(filtered_df)

    # Time series graph
    time_series_fig = px.line(filtered_df, x='Date', y='Sales_Kg', title='Sales Over Time')

    # Product sales graph
    product_sales_fig = px.bar(filtered_df, x='Product', y='Sales_Kg', title='Product Sales')

    # Sales locations map
    sales_locations_fig = px.scatter_mapbox(
        filtered_df, lat='Latitude', lon='Longitude', size='Sales_Kg', color='City',
        hover_name='SoldTo', title='Sales Locations',
        mapbox_style="open-street-map", zoom=3
    )

    return time_series_fig, product_sales_fig, sales_locations_fig

!ngrok authtoken 2gQdmNbVEf6opKAuJA7s2EufayA_7Pmt1GKP1QmJLv7XKr7N3

# Run the app with ngrok
public_url = ngrok.connect(8050)
print(f'Public URL: {public_url}')

app.run_server(port=8050)

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


PyngrokNgrokHTTPError: ngrok client exception, API returned 502: {"error_code":103,"status_code":502,"msg":"failed to start tunnel","details":{"err":"failed to start tunnel: Your account may not run more than 3 tunnels over a single ngrok agent session.\nThe tunnels already running on this session are:\ntn_2gQfHPTyEFDfEzuC1dKI1Nhdb3Q, tn_2gQfpVivjOCoNVeBcnePYCZGnO0, tn_2gQg9F4XRE9504fmzhPqMPN2Umi\n\r\n\r\nERR_NGROK_324\r\n"}}
